In [1]:
import pandas as pd
import json

# Create top 10 per country 

In [2]:
df = pd.read_csv('data/rankings.csv')

In [3]:
df.date = pd.to_datetime(df.date)
df['year'] = df['date'].dt.year


In [4]:
df.head()

,artist_id,country,date,chartType,artist_name,image_link,currentRank,previousRank,peakRank,peakDate,appearancesOnChart,consecutiveAppearancesOnChart,entryStatus,entryRank,entryDate,year
0,4q3ewBCX7sLwd24euuV69X,GLOBAL,2025-05-15,TOP_ARTIST,Bad Bunny,https://i.scdn.co/image/ab6761610000517481f47f...,1,1,1,2021-10-28,187,187,NO_CHANGE,2,2021-10-21,2025
1,06HL4z0CvFAxyc27GXpf02,GLOBAL,2025-05-15,TOP_ARTIST,Taylor Swift,https://i.scdn.co/image/ab67616100005174e672b5...,2,2,1,2021-11-11,187,187,NO_CHANGE,6,2021-10-21,2025
2,1Xyo4u8uXC1ZmMpatF05PJ,GLOBAL,2025-05-15,TOP_ARTIST,The Weeknd,https://i.scdn.co/image/ab676161000051749e5289...,3,4,1,2022-01-13,187,187,MOVED_UP,11,2021-10-21,2025
3,3TVXtAsR1Inumwj472S9r4,GLOBAL,2025-05-15,TOP_ARTIST,Drake,https://i.scdn.co/image/ab67616100005174429338...,4,3,1,2023-10-12,187,187,MOVED_DOWN,3,2021-10-21,2025
4,0ys2OFYzWYB5hRDLCsBqxt,GLOBAL,2025-05-15,TOP_ARTIST,Fuerza Regida,https://i.scdn.co/image/ab67616100005174158a3d...,5,7,5,2025-05-15,128,128,MOVED_UP,190,2022-12-08,2025


In [5]:
result = {}
date_format = '%Y-%m-%d'

grouped = df.groupby(['year', 'country'])

for (year, country), group in grouped:
    bySong = group.groupby('artist_name')
    year = int(year)

    top10 = group.groupby('artist_name').agg({
        'currentRank': 'mean',
        'artist_id': 'first',
        'image_link': 'first',
    }).reset_index().sort_values(by='currentRank').head(10)
    

    if year not in result:
        result[year] = {}
    if country not in result[year]:
        result[year][country] = {}
    rank = 1
    for i, track in top10.iterrows():
        rank = int(rank)
        result[year][country][rank] = {}
        result[year][country][rank]['name'] = track.artist_name
        result[year][country][rank]['image_link'] = track.image_link
        result[year][country][rank]['aggregated_rank'] = track.currentRank
        
        top1_evolution = group[group['artist_id'] == track.artist_id].set_index('date')
        top1_evolution.index = top1_evolution.index.strftime(date_format)
        result[year][country][rank]['evolution'] = top1_evolution['currentRank'].to_dict()
        
        rank += 1


with open('top10_artist_by_country.json', 'w') as f:
    json.dump(result, f, indent=2, )

# Top 10 song per year

In [6]:
df = pd.read_csv('data/song.csv')

In [7]:
df.date = pd.to_datetime(df.date)
df['year'] = df['date'].dt.year


In [8]:
df.head()

,currentRank,previousRank,peakRank,peakDate,appearancesOnChart,consecutiveAppearancesOnChart,rankingMetric,entryStatus,entryRank,entryDate,track_id,track_name,image_link,artists,release_date,country,date,chartType,year
0,1,1,1,2025-04-17,11,11,"{'value': '2807843', 'type': 'STREAMS'}",NO_CHANGE,165,2025-03-06,6iOndD4OFo7GkaDypWQIou,La Plena - W Sound 05,https://i.scdn.co/image/ab67616d00001e02474010...,"W Sound, Beéle, Ovy On The Drums",2025-02-19,AR,2025-05-15,TOP_TRACK,2025
1,2,-1,2,2025-05-15,1,1,"{'value': '2547989', 'type': 'STREAMS'}",NEW_ENTRY,2,2025-05-15,0vcuJjViqrFCCWUrjrZW1X,RAMEN PARA DOS,https://i.scdn.co/image/ab67616d00001e02127ac8...,"Maria Becerra, Paulo Londra, XROSS",2025-05-08,AR,2025-05-15,TOP_TRACK,2025
2,3,4,3,2025-05-15,5,5,"{'value': '2111021', 'type': 'STREAMS'}",MOVED_UP,100,2025-04-17,7dCGkDZgWIDSZyxInMkdIv,Pa las girlas,https://i.scdn.co/image/ab67616d00001e027ca9b0...,Mattei,2025-02-24,AR,2025-05-15,TOP_TRACK,2025
3,4,3,3,2025-04-03,18,18,"{'value': '1850117', 'type': 'STREAMS'}",MOVED_DOWN,103,2025-01-16,4DnrAI8WyUY6gkOwl8GlPN,capaz (merengueton),https://i.scdn.co/image/ab67616d00001e026abd0f...,"Alleh, Yorghaki",2024-12-24,AR,2025-05-15,TOP_TRACK,2025
4,5,6,1,2025-01-30,19,19,"{'value': '1849873', 'type': 'STREAMS'}",MOVED_UP,18,2025-01-09,2lTm559tuIvatlT1u0JYG2,BAILE INoLVIDABLE,https://i.scdn.co/image/ab67616d00001e02bbd45c...,Bad Bunny,2025-01-05,AR,2025-05-15,TOP_TRACK,2025


In [9]:
result = {}
grouped = df.groupby(['year', 'country'])

for (year, country, ), group in grouped:
    bySong = group.groupby('track_name')
    year = int(year)

    top10 = group.groupby('track_name').agg({
        'currentRank': 'mean',
        'track_id': 'first',
        'image_link': 'first',
    }).reset_index().sort_values(by='currentRank').head(10)
    

    if year not in result:
        result[year] = {}
    if country not in result[year]:
        result[year][country] = {}
    rank = 1
    for i, track in top10.iterrows():
        rank = int(rank)
        result[year][country][rank] = {}
        result[year][country][rank]['name'] = track.track_name
        result[year][country][rank]['image_link'] = track.image_link
        result[year][country][rank]['aggregated_rank'] = track.currentRank
        
        top1_evolution = group[group['track_id'] == track.track_id].set_index('date')
        top1_evolution.index = top1_evolution.index.strftime(date_format)
        result[year][country][rank]['evolution'] = top1_evolution['currentRank'].to_dict()
        
        rank += 1

with open('top10_track_by_country.json', 'w') as f:
    json.dump(result, f, indent=2, )